<a href="https://colab.research.google.com/github/marccasals98/PiuPiuNet/blob/main/Preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use this file to extract spectrograms from your training set
# which needs to be organized in subfolders representing class names.

import os
import time

import numpy as np
import cv2

from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/data/data.zip > /dev/null

### CONFIGURATION

In [ ]:
seed = np.random.RandomState(1337)

In [ ]:
########################  DATASET  ########################
# Path settings (train audio and xml files, specs, test audio files and json metadata)
TRAINSET_PATH = 'data/train/'
DATASET_PATH = os.path.join('preprocessedData/test', 'spec')
NOISE_PATH = os.path.join('preprocessedData/test', 'noise')
VALIDSET_PATH = 'data/validation/'
TESTSET_PATH = 'data/test/'

# Define if you want to 'copy', 'move' or 'symlink' audio files
# If you use 'symlink' make sure your OS does support symbolic links and define TRAINSET_PATH absolute
SORT_MODE = 'copy'

# Maximum number of classes to use (None = no limit)
MAX_CLASSES = None

# Use this whitelist to pre-select species; leave the list empty if you want to include all species
CLASS_WHITELIST = []

# If not sorted, using only a subset of classes (MAX_CLASSES) will select classes randomly
SORT_CLASSES_ALPHABETICALLY = False  

# Specify minimum and maximum amount of samples (specs) per class
MIN_SAMPLES_PER_CLASS = -1   # -1 = no minimum                                      
MAX_SAMPLES_PER_CLASS = None # None = no limit

# Specify the signal-to-noise interval you want to pick samples from (filename contains value)
S2N_INTERVAL = [50, 2500]

# Size of validation split (0.05 = 5%)
VAL_SPLIT = 0.05

######################  SPECTROGRAMS  ##################################################################
# Type of frequency scaling, mel-scale = 'melspec', linear scale = 'linear'
SPEC_TYPE = 'melspec'

# Sample rate for recordings, other sampling rates will force re-sampling
SAMPLE_RATE = 44100

# Specify min and max frequency for low and high pass
SPEC_FMIN = 500
SPEC_FMAX = 15000

# Define length of chunks for spec generation, overlap of chunks and chunk min length
SPEC_LENGTH = 1.0
SPEC_OVERLAP = 0.0
SPEC_MINLEN = 1.0

# Threshold for distinction between noise and signal
SPEC_SIGNAL_THRESHOLD = 0.0001

# Limit the amount of specs per class when extracting spectrograms (None = no limit)
MAX_SPECS_PER_CLASS = 1000

#########################  IMAGE  ######################################################################

# Number of channels
IM_DIM = 1

# Image size (width, height), should be the same as spectrogram shape
IM_SIZE = (256, 128)

# Resize mode, options are:
# 'crop': Crops from center of the image
# 'cropRandom': Crops from random position
# 'squeeze': Ignores aspect ratio when resizing
# 'fill': Fills with random noise to keep aspect ratio
RESIZE_MODE = 'squeeze'

# Normalization mode (values between -1 and 1)
ZERO_CENTERED_NORMALIZATION = True

# List of rejected specs, which we want to use as noise samples during augmentation
if os.path.exists(NOISE_PATH):
    NOISE_SAMPLES = [os.path.join(NOISE_PATH, s) for s in os.listdir(NOISE_PATH)]
else:
    NOISE_SAMPLES = []

# Image augmentation, uncomment to use; specify mode + value
IM_AUGMENTATION = {#'roll_h':0.5,                   # Horizontal roll
                   'roll_v':0.15,                   # Vertical roll
                   #'crop':[0.1, 0.0, 0.05, 0.0],   # Random crop - top, left, bottom, right
                   #'noise':0.05,                   # Gaussian noise
                   'add':NOISE_SAMPLES,             # List of specs to add to original sample
                   #'brightness':0.15,              # Adjust brightness
                   #'dropout':0.25,                 # Dropout single pixels
                   #'blackout':0.10,                # Dropout entire regions
                   #'blur':3,                       # Image blur
                   #'zoom':0.25,                    # Random zoom (equally cropping each side)
                   #'rotate':10,                    # Rotate by angle
                   #'multiply':0.25,                # Multiply pixel values
                   #'mean':True                     # Substract mean from image
                  }

# Maximum number of random augmentations per image
# Each try has 50% chance of success; we do not use duplicate augmentations
AUGMENTATION_COUNT = 2

# Probability for image augmentation
AUGMENTATION_PROBABILITY = 0.5

####################  STATS AND LOG  ######################

# Global vars
STATS = {}
DO_BREAK = False

# Options for log mode are 'all', 'info', 'progress', 'error', 'result'
LOG_MODE = 'all'


In [ ]:
# This files handles log messages with support for different log-modes.
# Author: Stefan Kahl, 2018, Chemnitz University of Technology

import sys
sys.path.append("..")

log = ''
def show(s, new_line=False):

    global log

    if isinstance(s, (list, tuple)):
        for i in range(len(s)):
            print(s[i])
            log += str(s[i])
            if i < len(s) - 1:
                log += ' '
    else:
        print(s)
        log += str(s)

    if new_line:
        print('')
        log += '\n'
    else:
        log += ' '
    

def i(s, new_line=True):

    if LOG_MODE in ['all', 'info']:
        show(s, new_line)

def p(s, new_line=True):

    if LOG_MODE in ['all', 'progress']:        
        show(s, new_line)            

def e(s, new_line=True):

    if LOG_MODE in ['all', 'error']:        
        show(s, new_line)       

def r(s, new_line=True):

    if LOG_MODE in ['all', 'result']:        
        show(s, new_line)       

def clear():

    global log
    log = ''

### AUDIO UTILITIES

In [ ]:
import librosa

def openAudioFile(path, sample_rate=44100, as_mono=True, mean_substract=False):
    
    # Open file with librosa (uses ffmpeg or libav)
    sig, rate = librosa.load(path, sr=sample_rate, mono=as_mono)

    # Noise reduction?
    if mean_substract:
        sig -= sig.mean()

    return sig, rate

def splitSignal(sig, rate, seconds, overlap, minlen):

    # Split signal with overlap
    sig_splits = []
    for i in range(0, len(sig), int((seconds - overlap) * rate)):
        split = sig[i:i + int(seconds * rate)]

        # End of signal?
        if len(split) < int(minlen * rate):
            break
        
        # Signal chunk too short?
        if len(split) < int(rate * seconds):
            split = np.hstack((split, np.zeros((int(rate * seconds) - len(split),))))
        
        sig_splits.append(split)

    return sig_splits

def melspec(sig, rate, shape=(128, 256), fmin=500, fmax=15000, normalize=True, preemphasis=0.95):

    # shape = (height, width) in pixels

    # Mel-Spec parameters
    SAMPLE_RATE = rate
    N_FFT = shape[0] * 8 # = window length
    N_MELS = shape[0]
    HOP_LEN = len(sig) // (shape[1] - 1)    
    FMAX = fmax
    FMIN = fmin

    # Preemphasis as in python_speech_features by James Lyons
    if preemphasis:
        sig = np.append(sig[0], sig[1:] - preemphasis * sig[:-1])

    # Librosa mel-spectrum
    melspec = librosa.feature.melspectrogram(y=sig, sr=SAMPLE_RATE, hop_length=HOP_LEN, n_fft=N_FFT, n_mels=N_MELS, fmax=FMAX, fmin=FMIN, power=1.0)
    
    # Convert power spec to dB scale (compute dB relative to peak power)
    melspec = librosa.amplitude_to_db(melspec, ref=np.max, top_db=80)

    # Flip spectrum vertically (only for better visialization, low freq. at bottom)
    melspec = melspec[::-1, ...]

    # Trim to desired shape if too large
    melspec = melspec[:shape[0], :shape[1]]

    # Normalize values between 0 and 1
    if normalize:
        melspec -= melspec.min()
        if not melspec.max() == 0:
            melspec /= melspec.max()
        else:
            mlspec = np.clip(melspec, 0, 1)

    return melspec.astype('float32')

def stft(sig, rate, shape=(128, 256), fmin=500, fmax=15000, normalize=True):

    # shape = (height, width) in pixels

    # STFT-Spec parameters
    N_FFT = int((rate * shape[0] * 2) / abs(fmax - fmin)) + 1
    P_MIN = int(float(N_FFT / 2) / rate * fmin) + 1
    P_MAX = int(float(N_FFT / 2) / rate * fmax) + 1    
    HOP_LEN = len(sig) // (shape[1] - 1)

    # Librosa stft-spectrum
    spec = librosa.core.stft(sig, hop_length=HOP_LEN, n_fft=N_FFT, window='hamm')

    # Convert power spec to dB scale (compute dB relative to peak power)
    spec = librosa.amplitude_to_db(librosa.core.magphase(spec)[0], ref=np.max, top_db=80)

    # Trim to desired shape using cutoff frequencies
    spec = spec[P_MIN:P_MAX, :shape[1]]

    # Flip spectrum vertically (only for better visialization, low freq. at bottom)
    spec = spec[::-1, ...]    

    # Normalize values between 0 and 1
    if normalize:
        spec -= spec.min()
        if not spec.max() == 0:
            spec /= spec.max()
        else:
            spec = np.clip(spec, 0, 1)    
    
    return spec.astype('float32')

def get_spec(sig, rate, shape, spec_type='linear', **kwargs):

    if spec_type.lower()== 'melspec':
        return melspec(sig, rate, shape, **kwargs)
    else:
        return stft(sig, rate, shape, **kwargs)

def signal2noise(spec):

    # Get working copy
    spec = spec.copy()

    # Calculate median for columns and rows
    col_median = np.median(spec, axis=0, keepdims=True)
    row_median = np.median(spec, axis=1, keepdims=True)

    # Binary threshold
    spec[spec < row_median * 1.25] = 0.0
    spec[spec < col_median * 1.15] = 0.0
    spec[spec > 0] = 1.0

    # Median blur
    spec = cv2.medianBlur(spec, 3)

    # Morphology
    spec = cv2.morphologyEx(spec, cv2.MORPH_CLOSE, np.ones((3, 3), np.float32))

    # Sum of all values
    spec_sum = spec.sum()

    # Signal to noise ratio (higher is better)
    try:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1] * spec.shape[2])
    except:
        s2n = spec_sum / (spec.shape[0] * spec.shape[1])

    return s2n

def specsFromSignal(sig, rate, shape, seconds, overlap, minlen, **kwargs):

    # Split signal in consecutive chunks with overlap
    sig_splits = splitSignal(sig, rate, seconds, overlap, minlen)

    # Extract specs for every sig split
    for sig in sig_splits:

        # Get spec for signal chunk
        spec = get_spec(sig, rate, shape, **kwargs)

        yield spec

def specsFromFile(path, rate, seconds, overlap, minlen, shape, start=-1, end=-1, **kwargs):

    # Open file
    sig, rate = openAudioFile(path, rate)

    # Trim signal?
    if start > -1 and end > -1:
        sig = sig[int(start * rate):int(end * rate)]
        minlen = 0

    # Yield all specs for file
    for spec in specsFromSignal(sig, rate, shape, seconds, overlap, minlen, **kwargs):
        yield spec
    

### IMAGE UTILITIES

In [ ]:
# This file includes basic functionality for image processing
# including i/o handling, image augmentation and model input pre-processing
# Author: Stefan Kahl, 2018, Chemnitz University of Technology

import sys
sys.path.append("..")

import copy

import numpy as np
import cv2

#Fixed random seed
RANDOM = seed

def resetRandomState():
    global RANDOM
    RANDOM = np.random.RandomState(1337)

########################## I/O ###########################
def openImage(path, im_dim=1):
    
    # Open image
    if im_dim == 3:
        img = cv2.imread(path, 1)
    else:
        img = cv2.imread(path, 0)
        
    # Convert to floats between 0 and 1
    img = np.asarray(img / 255., dtype='float32')    

    return img

def showImage(img, name='IMAGE', timeout=-1):

    cv2.imshow(name, img)
    cv2.waitKey(timeout)

def saveImage(img, path):

    cv2.imwrite(path, img)

#################### PRE-PROCESSING ######################
def normalize(img, zero_center=False):

    # Normalize
    if not img.min() == 0 and not img.max() == 0:
        img -= img.min()
        img /= img.max()
    else:
        img = img.clip(0, 1)

    # Use values between -1 and 1
    if zero_center:
        img -= 0.5
        img *= 2

    return img

def substractMean(img, clip=True):

    # Only suitable for RGB images
    if len(img.shape) == 3:

        # Normalized image?
        if img.max() <= 1.0:

            img[:, :, 0] -= 0.4850 #B
            img[:, :, 1] -= 0.4579 #G
            img[:, :, 2] -= 0.4076 #R

        else:

            img[:, :, 0] -= 123.680 #B
            img[:, :, 1] -= 116.779 #G
            img[:, :, 2] -= 103.939 #R
        
    else:
        img -= np.mean(img)

    # Clip values
    if clip:
        img = img.clip(0, img.max())

    return img

def prepare(img):

    # ConvNet inputs in Theano are 4D-vectors: (batch size, channels, height, width)
    
    # Add axis if grayscale image
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]

    # Transpose axis, channels = axis 0
    img = np.transpose(img, (2, 0, 1))

    # Add new dimension
    img = np.expand_dims(img, 0)

    return img

######################## RESIZING ########################
def resize(img, width, height, mode='squeeze'):

    if img.shape[:2] == (height, width):
        return img

    if mode == 'crop' or mode == 'cropCenter':
        img = cropCenter(img, width, height)
    elif mode == 'cropRandom':
        img = cropRandom(img, width, height)
    elif mode == 'fill':
        img = fill(img, width, height)
    else:
        img = squeeze(img, width, height)

    return img

def squeeze(img, width, height):

    # Squeeze resize: Resize image and ignore aspect ratio
    
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)

def cropRandom(img, width, height):

    # Random crop: Scale shortest side to minsize, crop with random offset

    # Original image shape
    h, w = img.shape[:2]
    aspect_ratio = float(max(h, w)) / float(min(h, w))

    # Scale original image
    minsize = int(max(width, height) * 1.1)
    if w <= h and w < minsize:
        img = squeeze(img, minsize, int(minsize * aspect_ratio))
    elif h < w and h < minsize:
        img = squeeze(img, int(minsize * aspect_ratio), minsize)

    #crop with random offset
    h, w = img.shape[:2]
    top = RANDOM.randint(0, h - height)
    left = RANDOM.randint(0, w - width)
    new_img = img[top:top + height, left:left + width]

    return new_img

def cropCenter(img, width, height):

    # Center crop: Scale shortest side, crop longer side

    # Original image shape
    h, w = img.shape[:2]
    aspect_ratio = float(max(h, w)) / float(min(h, w))

    # Scale original image
    if w == h:
        img = squeeze(img, max(width, height), max(width, height))
    elif width >= height:
        if h >= w:
            img = squeeze(img, width, int(width * aspect_ratio))
        else:            
            img = squeeze(img, int(height * aspect_ratio), height)
    else:
        if h >= w:
            img = squeeze(img, int(height / aspect_ratio), height)
        else:
            img = squeeze(img, int(height * aspect_ratio), height)

    #Crop from original image
    top = (img.shape[0] - height) // 2
    left = (img.shape[1] - width) // 2
    new_img = img[top:top + height, left:left + width]
    
    return new_img

def fill(img, width, height):

    # Fill mode: Scale longest side, pad shorter side with noise

    # Determine new shape
    try:
        new_shape = (height, width, img.shape[2])
    except:
        new_shape = (height, width)

    # Allocate array with noise
    new_img = RANDOM.normal(0.0, 1.0, new_shape)

    # Original image shape
    h, w = img.shape[:2]
    aspect_ratio = float(max(h, w)) / float(min(h, w))

    # Scale original image
    if w == h:
        img = squeeze(img, min(width, height), min(width, height))
    elif width >= height:
        if h >= w:
            img = squeeze(img, int(height / aspect_ratio), height)
        else:
            img = squeeze(img, width, int(width / aspect_ratio))
    else:
        if h >= w:
            img = squeeze(img, width, int(width * aspect_ratio))            
        else:
            img = squeeze(img, width, int(width / aspect_ratio))
            
    # Place original image at center of new image
    top = (height - img.shape[0]) // 2
    left = (width - img.shape[1]) // 2
    new_img[top:top + img.shape[0], left:left + img.shape[1]] = img
    
    return new_img

###################### AUGMENTATION ######################
def augment(img, augmentation={}, count=3, probability=0.5):

    # Make working copy
    augmentations = copy.deepcopy(augmentation)

    # Choose number of augmentations according to count
    # Count = 3 means either 0, 1, 2 or 3 different augmentations
    while(count > 0 and len(augmentations) > 0):

        # Roll the dice if we do augment or not
        if RANDOM.choice([True, False], p=[probability, 1 - probability]):

            # Choose one method
            aug = RANDOM.choice(augmentations.keys())
            
            # Call augementation methods
            if aug == 'flip':
                img = flip(img, augmentations[aug])
            elif aug == 'rotate':
                img = rotate(img, augmentations[aug])
            elif aug == 'zoom':
                img = zoom(img, augmentations[aug])
            elif aug == 'crop':
                if isinstance(augmentations[aug], float):
                    img = crop(img, top=augmentations[aug], left=augmentations[aug], right=augmentations[aug], bottom=augmentations[aug])
                else:
                    img = crop(img, top=augmentations[aug][0], left=augmentations[aug][1], bottom=augmentations[aug][2], right=augmentations[aug][3])
            elif aug == 'roll':
                img = roll(img, vertical=augmentations[aug], horizontal=augmentations[aug])
            elif aug == 'roll_v':
                img = roll(img, vertical=augmentations[aug], horizontal=0)
            elif aug == 'roll_h':
                img = roll(img, vertical=0, horizontal=augmentations[aug])
            elif aug == 'mean':
                img = mean(img, augmentations[aug])
            elif aug == 'noise':
                img = noise(img, augmentations[aug])
            elif aug == 'dropout':
                img = dropout(img, augmentations[aug])
            elif aug == 'blackout':
                img = blackout(img, augmentations[aug])
            elif aug == 'blur':
                img = blur(img, augmentations[aug])
            elif aug == 'brightness':
                img = brightness(img, augmentations[aug])
            elif aug == 'multiply':
                img = randomMultiply(img, augmentations[aug])
            elif aug == 'hue':
                img = hue(img, augmentations[aug])
            elif aug == 'lightness':
                img = lightness(img, augmentations[aug])
            elif aug == 'add':
                img = add(img, augmentations[aug])
            else:
                pass

            # Remove key so we avoid duplicate augmentations
            del augmentations[aug]
        
        # Count (even if we did not augment)
        count -= 1
    
    return img    
    
def flip(img, flip_axis=1):
    
    return cv2.flip(img, flip_axis)

def rotate(img, angle, zoom=1.0):

    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), RANDOM.uniform(-angle, angle), zoom)
    
    return cv2.warpAffine(img, M,(w, h))

def zoom(img, amount=0.33):

    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), 0, 1 + RANDOM.uniform(0, amount))
    
    return cv2.warpAffine(img, M,(w, h))

def crop(img, top=0.1, left=0.1, bottom=0.1, right=0.1):

    h, w = img.shape[:2]

    t_crop = max(1, int(h * RANDOM.uniform(0, top)))
    l_crop = max(1, int(w * RANDOM.uniform(0, left)))
    b_crop = max(1, int(h * RANDOM.uniform(0, bottom)))
    r_crop = max(1, int(w * RANDOM.uniform(0, right)))

    img = img[t_crop:-b_crop, l_crop:-r_crop]    
    img = squeeze(img, w, h)

    return img

def roll(img, vertical=0.1, horizontal=0.1):

    # Vertical Roll
    img = np.roll(img, int(img.shape[0] * RANDOM.uniform(-vertical, vertical)), axis=0)

    # Horizontal Roll
    img = np.roll(img, int(img.shape[1] * RANDOM.uniform(-horizontal, horizontal)), axis=1)

    return img

def mean(img, normalize=True):

    img = substractMean(img, True)

    if normalize and not img.max() == 0:
        img /= img.max()

    return img

def noise(img, amount=0.05):

    img += RANDOM.normal(0.0, RANDOM.uniform(0, amount**0.5), img.shape)
    img = np.clip(img, 0.0, 1.0)

    return img

def dropout(img, amount=0.25):

    d = RANDOM.uniform(0, 1, img.shape)
    d[d <= amount] = 0
    d[d > 0] = 1
    
    return img * d

def blackout(img, amount=0.25):

    b_width = int(img.shape[1] * amount)
    b_start = RANDOM.randint(0, img.shape[1] - b_width)

    img[:, b_start:b_start + b_width] = 0

    return img

def blur(img, kernel_size=3):

     return cv2.blur(img, (kernel_size, kernel_size))

def brightness(img, amount=0.25):

    img *= RANDOM.uniform(1 - amount, 1 + amount)
    img = np.clip(img, 0.0, 1.0)

    return img

def randomMultiply(img, amount=0.25):

    img *= RANDOM.uniform(1 - amount, 1 + amount, size=img.shape)
    img = np.clip(img, 0.0, 1.0)

    return img

def hue(img, amount=0.1):

    try:
        # Only works with BGR images
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hsv[:, :, 0] *= RANDOM.uniform(1 - amount, 1 + amount)
        hsv[:, :, 0].clip(0, 360)
        img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)        
    except:
        pass

    return img

def lightness(img, amount=0.25):

    try:
        # Only works with BGR images
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        lab[:, :, 0] *= RANDOM.uniform(1 - amount, 1 + amount)
        lab[:, :, 0].clip(0, 255)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    except:
        pass

    return img

def add(img, items):

    # Choose one item from List
    index = RANDOM.randint(len(items))

    # Open and resize image
    img2 = openImage(items[index], IM_DIM)
    img2 = resize(img2, img.shape[1], img.shape[0])

    # Generate random weights
    w1 = RANDOM.uniform(1, 2)
    w2 = RANDOM.uniform(1, 2)

    # Add images and calculate average
    img = (img * w1 + img2 * w2) / (w1 + w2)

    return img
    

### COMPUTE SPECTROGRAMS

In [ ]:
def getSpecs(path):
    
    specs = []
    noise = []

    # Get mel-specs for file
    for spec in specsFromFile(path,
                                    rate=SAMPLE_RATE,
                                    seconds=SPEC_LENGTH,
                                    overlap=SPEC_OVERLAP,
                                    minlen=SPEC_MINLEN,
                                    fmin=SPEC_FMIN,
                                    fmax=SPEC_FMAX,
                                    spec_type=SPEC_TYPE,
                                    shape=(IM_SIZE[1], IM_SIZE[0])):

        # Determine signal to noise ratio
        s2n = signal2noise(spec)
        specs.append(spec)
        noise.append(s2n)

    # Shuffle arrays (we want to select randomly later)
    specs, noise = shuffle(specs, noise, random_state=seed)

    return specs, noise

In [ ]:
def parseDataset():

    # List of classes, subfolders as class names
    CLASSES = [c for c in sorted(os.listdir(TESTSET_PATH))]

    # Parse every class
    for c in CLASSES:

        # List all audio files
        afiles = [f for f in sorted(os.listdir(os.path.join(TESTSET_PATH, c, 'Spain')))]
        afiles = afiles[0:-1] #remove .json located at the end

        # Calculate maximum specs per file
        max_specs = 20

        # Get specs for every audio file
        for idx in range(len(afiles)):

            spec_cnt = 0

            #try:

            # Stats
            print(idx + 1, '/', len(afiles), c, afiles[idx])

            # Get specs and signal to noise ratios
            specs, noise = getSpecs(os.path.join(TESTSET_PATH, c, 'Spain', afiles[idx]))

            # Save specs if it contains signal
            for s in range(len(specs)):

                # NaN?
                if np.isnan(noise[s]):
                    noise[s] = 0.0

                # Above SIGNAL_THRESHOLD?
                if noise[s] >= SPEC_SIGNAL_THRESHOLD:

                    # Create target path for accepted specs
                    filepath = os.path.join(DATASET_PATH, c)
                    if not os.path.exists(filepath):
                        os.makedirs(filepath)

                    # Count specs
                    spec_cnt += 1
                    print('Above SIGNAL_THRESHOLD: creating new spec file')

                else:

                    # Create target path for rejected specs -
                    # but we don't want to save every sample (only 10%)
                    if seed.choice([True, False], p=[0.1, 0.90]):
                        filepath = os.path.join(NOISE_PATH)
                        if not os.path.exists(filepath):
                            os.makedirs(filepath)
                        print('Below SIGNAL_THRESHOLD: creating new noise file')
                    else:
                        filepath = None
                        print('Below SIGNAL_THRESHOLD: not creating new noise file (10% probability)')
                
                if filepath:
                    # Filename contains s2n-ratio
                    filename =  afiles[idx].split('.')[0] + '_SpecIdx_' + str(s).zfill(3) + '_SNR_' + str(int(noise[s] * 10000)).zfill(4)

                    # Write to HDD
                    cv2.imwrite(os.path.join(filepath, filename + '.png'), specs[s] * 255.0)                        

                # Do we have enough specs already?
                if spec_cnt >= max_specs:
                    break

            # Stats
            i((spec_cnt, 'specs'))       

            #except:
            #    e((spec_cnt, 'specs', 'ERROR DURING SPEC EXTRACT'))
            #    continue


In [ ]:
parseDataset()

1 / 36 Cardueliscarduelis Cardueliscarduelis179322.mp3


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
20
specs

2 / 36 Cardueliscarduelis Cardueliscarduelis236446

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
20
specs

3 / 36 Cardueliscarduelis Cardueliscarduelis239679

...

Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
20
specs

11 / 12 Upupaepops Upupaepops627442.mp3


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
20
specs

12 / 12 Upupaepops Upupaepops73492.mp3


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
Above SIGNAL_THRESHOLD: creating new spec file
20
specs



In [ ]:
!zip -r data/preprocessedData_test.zip preprocessedData/test/

  adding: preprocessedData/test/ (stored 0%)
  adding: preprocessedData/test/spec/ (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/ (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops627442_SpecIdx_002_SNR_0865.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops278195_SpecIdx_016_SNR_0162.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops462754_SpecIdx_019_SNR_0223.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops291958_SpecIdx_000_SNR_0004.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops462754_SpecIdx_013_SNR_0511.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops73492_SpecIdx_018_SNR_1013.png (deflated 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops263714_SpecIdx_013_SNR_0092.png (stored 0%)
  adding: preprocessedData/test/spec/Upupaepops/Upupaepops73492_SpecIdx_003_SNR_0458.png (stored 0%)
  adding: preprocessedData/t

In [ ]:
import shutil
shutil.move("data/preprocessedData_test.zip", "gdrive/MyDrive/preprocessedData")

'gdrive/MyDrive/preprocessedData/preprocessedData_test.zip'